In [ ]:
### selenium 문법
0. find(단일 태그) / findall(복수 태그) // select(복수의 선택자: . #) / select_one(단일 선택자)
1. find_elements : 복수의 값을 찾아온다 -> 리스트 (동일한 요소를 전체 다 찾아서 가져온다)
2. find_element : 단수의 값을 찾아온다 -> 위에서 문서를 읽다가, 최초로 발견한 값을 찾아오고 종료
3. By : .TAG_NAME (태그를 중심), .ID (아이디중심) -> 문서 1번밖에 사용불가, .CSS_SELECTOR (선택자중심 -> 클래스, 태그, 아이디), .NAME / By.XPATH, Xpath 문법 활용 / By.CLASS_NAME
   
5. get_attribute() : 특정 속성의 속성값을 찾아오고자 할 때
6. .clear() : 특정 영역 내 값을 삭제 (Form 요소 기준) > imput[type=radio, check, tect], textarea 외
7. .send_keys() : 어떤 값 혹은 키보드의 키값을 전달
8. Keys : 키보드의 키를 입력하고자 할 때
9. sleep(N) : N초의 시간동안을 지연시키고자 할 때
10. screenshot(filename) : filename으로 스크린 캡처 가능
11. set_window_size(가로, 세로) : 윈도의 창을 가로 세로 사이즈로 조정

In [ ]:
# pseudo-code
# 무신사 사이트 접속
# 상단 검색바 클릭
# 인기 검색어 > 1등 키워드를 서칭
# 니트 키워드 입력 > 니트 상품 PLP
# 니트 관련 상품 목록 > 인기상품 5개 선정
# 인기 상품 5개 각각의 상세페이지 PDP 접속
# 각 상품별 리뷰(댓글) 목록 확인
# 해당 리뷰(댓글) 수집
# 엑셀 저장
# MySQL 저장

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

URL = "https://www.musinsa.com/main/musinsa/recommend?gf=A" 
service = Service(ChromeDriverManager().install())
options = Options()

# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR") 
options.add_argument("--no-sandbox") 

driver = webdriver.Chrome(service=service, options=options)
driver.get(URL)
time.sleep(3)

search_btn = driver.find_element(By.CLASS_NAME, "Searchbar__TextFieldWrap-sc-dzagxs-2")
search_btn.click()
time.sleep(2)

popular_keywords = driver.find_element(By.CLASS_NAME, "search-home-popular-wrap")
items = popular_keywords.find_elements(By.TAG_NAME, "li")
target_keyword = items[0].text  
time.sleep(2)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
search_input.send_keys(Keys.RETURN)

print("타겟키워드 : ", target_keyword)
print("페이지타이틀 : ", driver.title)

driver.quit()

타겟키워드 :  니트
페이지타이틀 :  니트 | 무신사 추천 상품


In [13]:
pip install pandas 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

URL = "https://www.musinsa.com/main/musinsa/recommend?gf=A" 
service = Service(ChromeDriverManager().install())
options = Options()

# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR") 
options.add_argument("--no-sandbox") 

driver = webdriver.Chrome(service=service, options=options)
driver.get(URL)
time.sleep(2)

search_btn = driver.find_element(By.CLASS_NAME, "Searchbar__TextFieldWrap-sc-dzagxs-2")
search_btn.click()
time.sleep(2)

popular_keywords = driver.find_element(By.CLASS_NAME, "search-home-popular-wrap")
items = popular_keywords.find_elements(By.TAG_NAME, "li")
target_keyword = items[0].text  
time.sleep(2)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
search_input.send_keys(Keys.RETURN)
time.sleep(2)

product_data = list()

for _ in range(2) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    products = driver.find_elements(By.CSS_SELECTOR, "div.sc-cpclqO.jrLXZA")[:5]
    for product in products :
        try : 
            product_info = product.find_element(By.CSS_SELECTOR, "div.sc-itBLYH.cDYUnS")
            brand = product_info.find_element(By.CSS_SELECTOR, "span.text-etc_11px_semibold").text.strip()
            name = product_info.find_element(By.CSS_SELECTOR, "span.text-body_13px_reg").text.strip()
            url = product_info.find_element(By.CSS_SELECTOR, "a.gtm-select-item").get_attribute("href")
            review_infos = product.find_elements(By.CSS_SELECTOR, "span.text-etc_11px_reg.text-yellow.font-pretendard")
            review_rating = float(review_infos[0].text.strip()) if len(review_infos) > 0 else ""
            review_count_before = review_infos[1].text.strip() if len(review_infos) > 1 else ""
            review_count = int(re.sub(r"[^\d]","", review_count_before))
            
            product_data.append({"브랜드명" : brand, "상품명" : name, "URL" : url, "평점" : review_rating, "리뷰수" : review_count})
        except Exception as e :
            print("데이터 크롤링 실패 :", e)
            continue

df = pd.DataFrame(product_data)
df_top10 = df.sort_values(by="리뷰수", ascending=False).head(10).reset_index(drop=True)

driver.quit()

df_top10

,브랜드명,상품명,URL,평점,리뷰수
0,엔티비씨,2Way 후드 패딩 아크틱 라이트웨이트 히트 점퍼,https://www.musinsa.com/products/5406681,4.7,64
1,디스이즈네버댓,Edelweiss Quilted Jacket Navy,https://www.musinsa.com/products/3591208,4.9,50
2,무신사 스탠다드 우먼,[시티 레저] 우먼즈 시어 립스탑 후디드 라이트 다운 재킷 [라이트 그레이],https://www.musinsa.com/products/5671194,4.6,44
3,플랙,후드 경량 다운 자켓_3colors,https://www.musinsa.com/products/5462546,4.7,34
4,칼라파타르,BACK LOGO DUCK DOWN LIGHTWEIGHT PADDED JACKET ...,https://www.musinsa.com/products/5634565,4.7,23
5,더콜디스트모먼트,TCM mini logo raglan light puffer jacket (white),https://www.musinsa.com/products/5558704,4.8,13
6,레테르코모,슈퍼라이트 인슐레이션 점퍼 점퍼 챠콜,https://www.musinsa.com/products/5431752,4.8,13
7,무드인사이드,미니멀 경량 후디드 패딩 자켓 _Olive,https://www.musinsa.com/products/5729108,4.2,9
8,네파,7K72060 남성 모노 튜브 스텐넥 다운 자켓,https://www.musinsa.com/products/4503753,5.0,4
9,무신사 스탠다드 스포츠,아이언 라이트 다운 자켓 (5color),https://www.musinsa.com/products/4306408,4.9,1


In [49]:
from openpyxl import Workbook

driver = webdriver.Chrome(service=service, options=options)
all_reviews = list()

for idx, row in df_top10.iterrows():
    print(f"{idx + 1}번째 상품리뷰 수집 중:", row["상품명"])
    driver.get(row["URL"])
    time.sleep(2)

    try :
        review_tab = driver.find_element(By.CSS_SELECTOR, "button[data-button-id='prd_review_tab']")
        review_tab.click()
        time.sleep(5)

        review_elements = driver.find_elements(By.CSS_SELECTOR, "div.ExpandableContent__Container-sc-1951ro-0 > div.ExpandableContent__ContentContainer-sc-1951ro-1 span.text-body_13px_reg.w-full.text-black.font-pretendard")
        for review in review_elements :

            text = (review.text or "").strip()
            if not text :
                continue
           
            all_reviews.append({
                "브랜드" : row["브랜드명"],
                "상품명" : row["상품명"],
                "URL" : row["URL"],
                "리뷰수" : row["리뷰수"],
                "리뷰" : review.text.strip(),
                "평점" : row["평점"]
            })
    except Exception as e :
        print("리뷰 수집 실패 :", e)
        continue

driver.quit()

wb = Workbook()
ws = wb.active
ws.title = "무신사 리뷰"
ws.append(["브랜드", "상품명", "URL", "리뷰수", "리뷰", "평점"])

for item in all_reviews :
    ws.append([item["브랜드"], item["상품명"], item["URL"], item["리뷰수"], item["리뷰"], item["평점"]])

file_name = "musinsa_top10_reviews.xlsx"
wb.save(file_name)    
print("엑셀 파일 저장완료:", file_name)

1번째 상품리뷰 수집 중: 2Way 후드 패딩 아크틱 라이트웨이트 히트 점퍼
2번째 상품리뷰 수집 중: Edelweiss Quilted Jacket Navy
3번째 상품리뷰 수집 중: [시티 레저] 우먼즈 시어 립스탑 후디드 라이트 다운 재킷 [라이트 그레이]
4번째 상품리뷰 수집 중: 후드 경량 다운 자켓_3colors
5번째 상품리뷰 수집 중: BACK LOGO DUCK DOWN LIGHTWEIGHT PADDED JACKET -BLACK
6번째 상품리뷰 수집 중: TCM mini logo raglan light puffer jacket (white)
7번째 상품리뷰 수집 중: 슈퍼라이트 인슐레이션 점퍼 점퍼 챠콜
8번째 상품리뷰 수집 중: 미니멀 경량 후디드 패딩 자켓 _Olive
9번째 상품리뷰 수집 중: 7K72060 남성 모노 튜브 스텐넥 다운 자켓
10번째 상품리뷰 수집 중: 아이언 라이트 다운 자켓 (5color)
엑셀 파일 저장완료: musinsa_top10_reviews.xlsx


In [69]:
import pandas as pd
import re
import hashlib
from sqlalchemy import create_engine, text

EXCEL_PATH = "musinsa_top10_reviews.xlsx"
HOST = "127.0.0.1"
USER = "root"
PASSWORD = "Ye916154&"
PORT = 3306
DB = "musinsa_db"

df = pd.read_excel(EXCEL_PATH)

ENGINE = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}?charset=utf8mb4", pool_pre_ping=True, pool_recycle=3600)

df.columns = [str(c).strip() for c in df.columns] # [브랜드, 상품명, URL, 리뷰수, 리뷰, 평점]
needed = ["브랜드", "상품명", "URL", "리뷰수", "리뷰", "평점"]
missing = [c for c in needed if c not in df.columns]

if missing :
    raise ValueError(f"액셀에서 해당 컬럼을 찾지 못했습니다. : {missing}")

def extract_product_id(url: str) -> int | None :
    if not url :
        return None
    m = re.search(r"/products/(\d+)", str(url))
    return int(m.group(1)) if m else None    

def sha1_text(s: str) -> str : 
    s = (s or "").strip()
    return hashlib.sha1(s.encode("utf-8")).hexdigest()
    

out = pd.DataFrame()
out["brand"] = df["브랜드"].astype(str).str.strip()
out["product_name"] = df["상품명"].astype(str).str.strip()
out["url"] = df["URL"].astype(str).str.strip()
out["product_id"] = out["url"].apply(extract_product_id)
out["review_count"] = df["리뷰수"]
out["rating"] = df["평점"]
out["review_text"] = (
    df["리뷰"].astype(str)
    .str.replace("\u200b", "", regex=False) # white space 없애기 -> ex) 엔터키
    .str.replace("\xa0", "", regex=False) # escape sequence 값을 여백 1개로 줄이기
    .str.replace(r"\s+", " ", regex=True) # 정규표현식으로 여러개 공백을 1개로 줄이기
    .str.strip()
)
out["review_hash"] = out["review_text"].apply(sha1_text)
out["source_file"] = EXCEL_PATH
out = out[out["review_text"].notna() & (out["review_text"] != "")]
out = out.drop_duplicates(subset=["review_hash"])

INSERT_SQL = text(""" 
INSERT INTO musinsa_reviews 
(brand, product_name, product_id, url, review_count, rating, review_text, review_hash, source_file)
VALUES
(:brand, :product_name, :product_id, :url, :review_count, :rating, :review_text, :review_hash, :source_file)
ON DUPLICATE KEY UPDATE
    rating = VALUES(rating),
    review_count = VALUES(review_count),
    product_name = VALUES(product_name),
    brand = VALUES(brand),
    url = VALUES(url),
    source_file = VALUES(source_file);
""")

def chunked(iterable, size: int) : 
    for i in range(0, len(iterable), size) :
        yield iterable[i:i+size]

records = out.to_dict(orient="records")

CHUNK_SIZE = 2000
with ENGINE.begin() as conn :
    for batch in chunked(records, CHUNK_SIZE) :
        conn.execute(INSERT_SQL, batch)

print(f"완료 : {len(records)}건 처리 완료")

완료 : 89건 처리 완료
